In [1]:
import zipfile
import pandas as pd
import nltk
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from stop_words import get_stop_words
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\frede\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\frede\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\frede\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
archive = zipfile.ZipFile('training_data.zip', 'r')
df = pd.read_csv(archive.open('training.1600000.processed.noemoticon.csv'), header=None, encoding_errors="replace")
# df = pd.read_csv("training_data/training.1600000.processed.noemoticon.csv", encoding="utf-8", encoding_errors="replace")

In [3]:
# target: the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)

# ids: The id of the tweet ( 2087)

# date: the date of the tweet (Sat May 16 23:58:44 UTC 2009)

# flag: The query (lyx). If there is no query, then this value is NO_QUERY.

# user: the user that tweeted (robotickilldozr)

# text: the text of the tweet (Lyx is cool)

df.loc[df[0]==4, 0]= 1
df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
data = df.drop([1,2,3,4], axis=1)
# data.columns = ["label","text"]
data.head()

,0,5
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [5]:
# data = data["text"].dropna()
data[5] = [entry.lower() for entry in data[5]]
data[5] = data[5].astype("string")


In [6]:
# Import stopwords with nltk.
from nltk.corpus import stopwords
stop = stopwords.words('english')

pos_tweets = data

test = pd.DataFrame(pos_tweets)
test.columns = ["class","tweet"]

# Exclude stopwords with Python's list comprehension and pandas.DataFrame.apply.
test['tweet_without_stopwords'] = test['tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
print(test)

         class                                              tweet  \
0            0  @switchfoot http://twitpic.com/2y1zl - awww, t...   
1            0  is upset that he can't update his facebook by ...   
2            0  @kenichan i dived many times for the ball. man...   
3            0    my whole body feels itchy and like its on fire    
4            0  @nationwideclass no, it's not behaving at all....   
...        ...                                                ...   
1599995      1  just woke up. having no school is the best fee...   
1599996      1  thewdb.com - very cool to hear old walt interv...   
1599997      1  are you ready for your mojo makeover? ask me f...   
1599998      1  happy 38th birthday to my boo of alll time!!! ...   
1599999      1  happy #charitytuesday @thenspcc @sparkscharity...   

                                   tweet_without_stopwords  
0        @switchfoot http://twitpic.com/2y1zl - awww, t...  
1        upset can't update facebook texting it..

In [7]:
data[5] = test['tweet_without_stopwords'].values
data[5]

0          @switchfoot http://twitpic.com/2y1zl - awww, t...
1          upset can't update facebook texting it... migh...
2          @kenichan dived many times ball. managed save ...
3                           whole body feels itchy like fire
4          @nationwideclass no, behaving all. i'm mad. he...
                                 ...                        
1599995                    woke up. school best feeling ever
1599996    thewdb.com - cool hear old walt interviews! ♫ ...
1599997                     ready mojo makeover? ask details
1599998    happy 38th birthday boo alll time!!! tupac ama...
1599999    happy #charitytuesday @thenspcc @sparkscharity...
Name: 5, Length: 1600000, dtype: object

In [8]:
# data[5]= [word_tokenize(entry) for entry in data[5]]

In [9]:
# data.to_csv('tokenized_data.csv', index=False)
data = pd.read_csv('tokenized_data.csv')
# data["5"] = data["5"].astype("string")
# data["5"].dtype

In [10]:
data

,class,tweet,tweet_without_stopwords,5
0,0,"@switchfoot http://twitpic.com/2y1zl - awww, t...","@switchfoot http://twitpic.com/2y1zl - awww, t...","['@', 'switchfoot', 'http', ':', '//twitpic.co..."
1,0,is upset that he can't update his facebook by ...,upset can't update facebook texting it... migh...,"['upset', 'ca', ""n't"", 'update', 'facebook', '..."
2,0,@kenichan i dived many times for the ball. man...,@kenichan dived many times ball. managed save ...,"['@', 'kenichan', 'dived', 'many', 'times', 'b..."
3,0,my whole body feels itchy and like its on fire,whole body feels itchy like fire,"['whole', 'body', 'feels', 'itchy', 'like', 'f..."
4,0,"@nationwideclass no, it's not behaving at all....","@nationwideclass no, behaving all. i'm mad. he...","['@', 'nationwideclass', 'no', ',', 'behaving'..."
...,...,...,...,...
1599995,1,just woke up. having no school is the best fee...,woke up. school best feeling ever,"['woke', 'up', '.', 'school', 'best', 'feeling..."
1599996,1,thewdb.com - very cool to hear old walt interv...,thewdb.com - cool hear old walt interviews! ♫ ...,"['thewdb.com', '-', 'cool', 'hear', 'old', 'wa..."
1599997,1,are you ready for your mojo makeover? ask me f...,ready mojo makeover? ask details,"['ready', 'mojo', 'makeover', '?', 'ask', 'det..."
1599998,1,happy 38th birthday to my boo of alll time!!! ...,happy 38th birthday boo alll time!!! tupac ama...,"['happy', '38th', 'birthday', 'boo', 'alll', '..."


In [11]:
data["5"].dtype

dtype('O')

In [12]:
data.head()

,class,tweet,tweet_without_stopwords,5
0,0,"@switchfoot http://twitpic.com/2y1zl - awww, t...","@switchfoot http://twitpic.com/2y1zl - awww, t...","['@', 'switchfoot', 'http', ':', '//twitpic.co..."
1,0,is upset that he can't update his facebook by ...,upset can't update facebook texting it... migh...,"['upset', 'ca', ""n't"", 'update', 'facebook', '..."
2,0,@kenichan i dived many times for the ball. man...,@kenichan dived many times ball. managed save ...,"['@', 'kenichan', 'dived', 'many', 'times', 'b..."
3,0,my whole body feels itchy and like its on fire,whole body feels itchy like fire,"['whole', 'body', 'feels', 'itchy', 'like', 'f..."
4,0,"@nationwideclass no, it's not behaving at all....","@nationwideclass no, behaving all. i'm mad. he...","['@', 'nationwideclass', 'no', ',', 'behaving'..."


In [13]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(data['5'],data['class'],test_size=0.20, random_state=10)

In [14]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(data['5'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [15]:
# print(Tfidf_vect.vocabulary_)

In [16]:
# print(Train_X_Tfidf)

In [17]:
# Naive Bayes Classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)

predictions_NB = Naive.predict(Test_X_Tfidf)
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  75.5721875


In [33]:
# Support Vector Machine Classifier
SVM = svm.SVC(C=0.9, kernel='rbf', degree=3, gamma='auto', cache_size= 1000, max_iter=2500,decision_function_shape="ovo", random_state=10)
SVM.fit(Train_X_Tfidf,Train_Y)

predictions_SVM = SVM.predict(Test_X_Tfidf)
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

c:\Users\frede\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


SVM Accuracy Score ->  51.4578125
